#### Configura acesso do jupyter notebook ao ambiente do Django

In [1]:
REDITUS = '/Users/mac/codes/reditus/reditus'
import os, sys
sys.path.insert(0, REDITUS)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "/reditus/settings.py")
import django
django.setup()

#### Importa bibliotecas necessárias

In [2]:
from search.models import SimilarSentence, Process
import pandas as pd

#### define formato de porcentagem

In [3]:
def percentage(similarity):
    return '{0:.2%}'.format(similarity)

##### Transforma as ProcessContestations em dataframe

In [4]:
sentences_ordered_qs = SimilarSentence.objects.order_by('similarity').reverse()

In [5]:
maintained_sentences_ordered_qs = sentences_ordered_qs.filter(similar_process__status=Process.Status.NAO_RECORRIDA)
len(maintained_sentences_ordered_qs)

684672

In [6]:
len(sentences_ordered_qs)

803313

In [6]:
sentences_ordered = [ (str(s.processcontestation.process.id),str(s.processcontestation.process.cod_cnj), str(s.similar_process.id), str(s.similar_process.cod_cnj), percentage(s.similarity)) for s in sentences_ordered_qs ]

#### Cria e ajusta dataframe

In [7]:
df = pd.DataFrame(sentences_ordered,columns=['COD_PROCESSO_A','COD_CNJ_A','COD_PROCESSO_B','COD_CNJ_B','SIMILARIDADE'])
df = df.drop_duplicates(subset=['COD_PROCESSO_A','COD_PROCESSO_B'], keep='first', inplace=False)
df = df.reset_index(drop=True)

#### gera csv

In [ ]:
df.to_csv('processos_similares.csv',sep=';')

In [9]:
writer = pd.ExcelWriter('processos_similares.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [7]:
len(Process.objects.filter(status=Process.Status.NAO_RECORRIDA))

72996

In [15]:
processes = [ (p.id, p.cod_cnj, p.comarca, p.serventia, p.status) for p in Process.objects.all().order_by('status','comarca','serventia').reverse() ]

In [19]:
processes_DF = pd.DataFrame(processes,columns=['COD_PROC','COD_CNJ','COMARCA','SERVENTIA','STATUS'])

In [26]:
processes_DF.to_csv('processos-status.csv',sep=';', encoding='utf_8')

In [28]:
writer = pd.ExcelWriter('processos-status.xlsx')
processes_DF.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
api.res